In [ ]:
import io
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
def WebScraping(url, tag_html, class_html):

    # requisição html
    response = requests.get(url)

    # Verifica solicitação bem-sucedida
    if response.status_code == 200:

        # Cria um objeto BeautifulSoup para analisar o HTML
        soup = BeautifulSoup(response.content, 'html.parser')
        # Encontra tabelas na página html
        tables = soup.find_all(tag_html, class_ = class_html)

        return tables
    else:
        print(f"Erro ao obter a página. Código de status: {response.status_code}")

In [ ]:
def FormataDataFrame(df):

    try:

        # salva o nome do dataframe
        df_name = df.columns.get_level_values(0)[0]
        # gera o conteúdo da coluna Tabela_origem
        col_name_table = [df_name] * len(df)
        # elimina a coluna multindex
        df = df.droplevel(level=0, axis=1)
        # insere a coluna Tabela_origem no dataframe
        df['Tabela_origem'] = col_name_table

        # filtrar colunas de interesse
        columns = ['Tabela_origem', 'Mercadoria', 'Sentido', 'Chegada', 'Previsto']
        df = df[columns]
        df = df.copy()

    except KeyError as error:
        pass

    else:
        ### trata a coluna Chegada

        # elimina espaços em branco, preenche nulos, elimina caracteres
        df['Previsto'] = df['Previsto'].str\
                                  .strip()\
                                  .fillna("0,0")\
                                  .replace(['Tons.', 'Movs.'], ['', ''], regex=True)
        # altera pontuação e tipo para float
        df['Previsto'] = df['Previsto'].str\
                                    .replace('.', '', regex=True)\
                                    .replace(',', '.', regex=True)\
                                    .astype(float)


        ### trata a coluna Chegada

        # converte a coluna para datetime
        df['Chegada'] = pd.to_datetime(df['Chegada'])
        # formata a coluna para o novo formato 'yyyyMMdd'
        df['Chegada'] = df['Chegada'].dt.strftime('%Y%m%d')


        ### soma as quantidades de mercadoria

        df_form = df[columns].groupby(['Mercadoria', 'Sentido', 'Tabela_origem', 'Chegada'])['Previsto'].sum().reset_index()
        df_form = df_form.sort_values(by='Chegada')
        return df_form

In [ ]:
def GeraDataFrames(url, tag_html, class_html):

    # EXTRAI AS TABELAS DO SITE
    tables = WebScraping(url, tag_html, class_html)


    dfs = []
    for table in tables:

        # CONVERTE A TABELA EM UM DATAFRAME
        df = pd.read_html(str(table))[0]

        # FORMATA O DATAFRAME
        df_form = FormataDataFrame(df)

        # ADICIONA O DATAFRAME À LISTA
        dfs.append(df_form)


    # JUNTA TODOS OS DATAFRAMES
    dfs_join = pd.concat(dfs).reset_index().drop(['index'], axis=1)


    # RETORNA O DATAFRAME
    return dfs_join



In [ ]:
# PARÂMETROS
url = 'https://www.appaweb.appa.pr.gov.br/appaweb/pesquisa.aspx?WCI=relLineUpRetroativo'
tag_html = "table"
class_html = 'table table-bordered table-striped table-hover'


# EXECUTA A FUNÇÃO
df = GeraDataFrames(url, tag_html, class_html)

# IMPRIME O DATAFRAME
df

,Mercadoria,Sentido,Tabela_origem,Chegada,Previsto
0,SOJA,Exp,ATRACADOS,20230209,69000.0
1,SULFATO DE AMONIO,Imp,ATRACADOS,20230712,67295.0
2,UREIA,Imp,ATRACADOS,20230911,37100.0
3,METANOL,Imp,ATRACADOS,20230912,9532.0
4,FARELO DE SOJA,Exp,ATRACADOS,20231026,60000.0
...,...,...,...,...,...
95,CONTÊINERES,Imp/Exp,DESPACHADOS,20231215,643.0
96,CONTÊINERES,Imp/Exp,DESPACHADOS,20231216,1358.0
97,OLEO DIESEL,Imp,DESPACHADOS,20231216,29819.0
98,AUTOMÓVEIS DE PASSAGEIROS E OU,Imp/Exp,DESPACHADOS,20231216,344.0


In [ ]:


# SALVA ARQUIVO
df.to_csv('/content/drive/MyDrive/desafioVeeries/lineup_porto_paranagua.csv',
    mode= 'x',
    sep= ';',
    decimal= '.',
    header= True
)

